In [34]:
#mount to google drive
from google.colab import drive
drive.mount('/content/drive')

%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Baseline_results.ipynb                           CRF_Training.ipynb
 BERT_Text_pre_processing.ipynb                   CRF_utils.py
 bert_text_pre_processing.py                      Embeddings_kmeans.ipynb
 BERT_Text_pre_processing_Teona_debugging.ipynb   MARTA_Bert_embeddings.ipynb
 BERT_Text_pre_processing_Teona_v.ipynb           MARTA_Teona_copy.ipynb
'Copy of BERT_Text_pre_processing.ipynb'          Preprocessing_fuction.ipynb
'copy of Embeddings_kmeans giomaria.ipynb'        __pycache__/
 Creating_Dataset.ipynb                           Teona_BERT_Model.ipynb


In [35]:
%cd drive/MyDrive/NLP Project/Code

[Errno 2] No such file or directory: 'drive/MyDrive/NLP Project/Code'
/content/drive/MyDrive/NLP Project/Code


In [36]:
!pip install rank_bm25
!pip install stop_words
!pip install transformers
!pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import re
import nltk
import pickle
import sklearn
import scipy.stats
import numpy as np
import pandas as pd
from itertools import chain
from gensim.models import Word2Vec
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics


nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
from bert_text_pre_processing import add_labels
from CRF_utils import sent2features

In [39]:
# Load word2Vec and kmeans
model = Word2Vec.load("/content/drive/MyDrive/NLP Project/Models/word2vec.model")
kmeans = pickle.load(open("/content/drive/MyDrive/NLP Project/Models/model.pkl", "rb"))

word_vectors = model.wv

vocab = np.array(list(model.wv.key_to_index.keys()))
word_vecs = []

for word in vocab:
    word_vecs.append(word_vectors[word])
    
word_array = np.array(word_vecs)

In [40]:
df_1 = pd.read_csv(r'/content/drive/MyDrive/NLP Project/Data/Combined Datasets/combined_df_1.csv')
df_2 = pd.read_csv(r'/content/drive/MyDrive/NLP Project/Data/Combined Datasets/combined_df_2.csv')

pre_processed = add_labels(df_1, df_2, 'other', 'text', 'txt_id', 'symptom', False)

# split dataset into training and test/val
np.random.seed(100)
train_df, valid_df = train_test_split(pre_processed, test_size=0.2)

train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)

In [41]:
train_df.head()

,txt_id,text,dataset,tokenized,pre_processed_tokens
0,VOLTAREN-XR.5,"Always tired, and possible blood clots. I was ...",CADEC,"[always, tired, and, possible, blood, clot, i,...","[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,cymbalta.13,"nonstop headache, constipation, racing thought...",PsyTar,"[nonstop, headache, constipation, racing, thou...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ..."
2,ARTHROTEC.109,Upset stomach and the feeling that I may need ...,CADEC,"[upset, stomach, and, the, feeling, that, i, m...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, ..."
3,176,too many to list but new problem 3 yrs after T...,Annotated_dataset,"[too, many, to, list, but, new, problem, yr, a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
4,ARTHROTEC.52,Went to hospital for my Tmj pain because it wa...,CADEC,"[went, to, hospital, for, my, tmj, pain, becau...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [42]:
# Extract features for the whole training set. Takes 2 minutes. 
train_sentences, train_labels = [], []
val_sentences, val_labels = [], []

for sent, lab in zip(train_df.tokenized.to_list(), train_df.pre_processed_tokens.to_list()):
  if len(sent) >= 6 and len(lab) == len(sent):
    train_sentences.append(sent)
    train_labels.append(lab)

for sent, lab in zip(valid_df.tokenized.to_list(), valid_df.pre_processed_tokens.to_list()):
  if len(sent) >= 6 and len(lab) == len(sent):
    val_sentences.append(sent)
    val_labels.append(lab)


print(train_sentences[0])
print(train_labels[0])

['always', 'tired', 'and', 'possible', 'blood', 'clot', 'i', 'wa', 'on', 'voltaren', 'for', 'about', 'year', 'and', 'all', 'of', 'the', 'sudden', 'had', 'a', 'minor', 'stroke', 'and', 'had', 'blood', 'clot', 'that', 'traveled', 'to', 'my', 'eye', 'i', 'had', 'every', 'test', 'in', 'the', 'book', 'done', 'at', 'the', 'hospital', 'and', 'they', 'couldnt', 'find', 'anything', 'i', 'wa', 'completley', 'healthy', 'i', 'am', 'thinking', 'it', 'wa', 'from', 'the', 'voltaren', 'i', 'have', 'been', 'off', 'of', 'the', 'drug', 'for', 'month', 'now', 'and', 'have', 'never', 'felt', 'better', 'i', 'started', 'eating', 'healthy', 'and', 'working', 'out', 'and', 'that', 'ha', 'help', 'alot', 'i', 'can', 'now', 'sleep', 'all', 'thru', 'the', 'night', 'i', 'wont', 'take', 'this', 'again', 'if', 'i', 'have', 'the', 'back', 'pain', 'i', 'will', 'pop', 'a', 'tylonol', 'instead']
[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [10]:
X_train = [sent2features(s, vocab,  kmeans.labels_) for s in train_sentences]
print("Done with pre-processing of train set")

X_val = [sent2features(s, vocab,  kmeans.labels_) for s in val_sentences]
print("Done with pre-processing of val set")

KeyboardInterrupt: ignored

In [ ]:
# Check format
print(X_train[0])
print(X_val[0])

[{'word': 'always', 'wordC': 97, 'word1': 'tired', 'word2': 'and', 'word3': 'possible', 'word3C': 135, 'word2C': 51, 'word1C': 48, 'BOS': True}, {'word': 'tired', 'wordC': 48, 'word1': 'and', 'word2': 'possible', 'word3': 'blood', 'word3C': 43, 'word2C': 135, 'word1C': 51, 'wordm1': 'always', 'wordm1C': 97}, {'word': 'and', 'wordC': 51, 'word1': 'possible', 'word2': 'blood', 'word3': 'clot', 'word3C': 7, 'word2C': 43, 'word1C': 135, 'wordm1': 'tired', 'wordm1C': 48, 'wordm2': 'always', 'wordm2C': 97}, {'word': 'possible', 'wordC': 135, 'wordm3': 'always', 'wordm2': 'tired', 'wordm1': 'and', 'word1': 'blood', 'word2': 'clot', 'word3': 'i', 'wordm3C': 97, 'wordm2C': 48, 'wordm1C': 51, 'word3C': 51, 'word2C': 7, 'word1C': 43}, {'word': 'blood', 'wordC': 43, 'wordm3': 'tired', 'wordm2': 'and', 'wordm1': 'possible', 'word1': 'clot', 'word2': 'i', 'word3': 'wa', 'wordm3C': 48, 'wordm2C': 51, 'wordm1C': 135, 'word3C': 64, 'word2C': 51, 'word1C': 7}, {'word': 'clot', 'wordC': 7, 'wordm3': 'and

In [ ]:
# Save files
with open("/content/drive/MyDrive/NLP Project/Data/CRF_Data/xTrain", "wb") as f:
    pickle.dump(X_train, f)
with open("/content/drive/MyDrive/NLP Project/Data/CRF_Data/xVal", "wb") as f:
    pickle.dump(X_val, f)

In [43]:
# Read files
X_train = pd.read_pickle(r"/content/drive/MyDrive/NLP Project/Data/CRF_Data/xTrain")
X_val = pd.read_pickle(r"/content/drive/MyDrive/NLP Project/Data/CRF_Data/xVal")

In [44]:
# Need to add this because I need the data type of the labels to be a string
# Plus, for some reason, some labels are longer than the sentences
y_train_CRF, y_val_CRF = [], []
for labTr, labVal in zip(train_labels, val_labels):
  y_train_CRF.append(np.array(labTr, dtype = 'str').tolist())
  y_val_CRF.append(np.array(labVal, dtype = 'str').tolist())
  

In [45]:
print(y_train_CRF[0])
print(X_train[0])

['0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
[{'word': 'always', 'wordC': 97, 'word1': 'tired', 'word2': 'and', 'word3': 'possible', 'word3C': 135, 'word2C': 51, 'word1C': 48, 'BOS': True}, {'word': 'tired', 'wordC': 48, 'word1': 'and', 'word2': 'possible', 'word3': 'blood', 'word3C': 43, 'word2C': 135, 'word1C': 51, 'wordm1': 'always', 'wordm1C': 97}, {'word': 'and', 'wordC': 51, 'word1': 'possible', 'word2': 'blood', 'word3': 'clot', 'word3C': 7, 'word2C': 43, 'word1C': 135, 'wordm1

In [46]:
# Train CRF
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)


try:
    crf.fit(X_train, y_train_CRF)
except AttributeError:
    pass

In [47]:
y_pred = crf.predict(X_val)

#y_pred_true = y_pred[(y_val_CRF == 0) | (y_val_CRF == 1)]
#y_val_CRF_true = y_val_CRF[(y_val_CRF == 0) | (y_val_CRF == 1)]

acc = metrics.flat_accuracy_score(y_val_CRF, y_pred)

recall = metrics.flat_recall_score(y_val_CRF, y_pred, pos_label='1')

precision = metrics.flat_precision_score(y_val_CRF, y_pred, pos_label='1')

F1 = metrics.flat_f1_score(y_val_CRF, y_pred, pos_label='1')

print(f"Accuracy: {acc}.")
print(f"Recall: {recall}.")
print(f"Precision: {precision}.")
print(f"F1: {F1}.")

Accuracy: 0.8933162939297125.
Recall: 0.5851565129585998.
Precision: 0.6704589278827613.
F1: 0.6249101365923796.
